# BOW
- 문맥, 순서를 무시하고 단어의 빈도 값으로 피처 값을 추출하는 모델

## 알고리즘
1. 각 문장에 있는 모든 단어에서 중복을 제거하고 각 단어를 칼럼 형태로 나열한 후 각 단어에 고유 인덱스 부여
2. 개별 문장에서 해당 단어가 나타나는 횟수(Occurrence)를 각 단어 인덱스에 기재

## 장점
- 쉽고 빠른 구축
- 문서의 특성을 잘 나타낸다는 경험으로 활용도가 높음

## 단점
- 문맥 의미(Semantic Context) 반영 부족
    - 단어의 순서를 고려하지 않음
    - n_gram 기법을 활용할 수 있으나 제한적임
- 희소 행렬 문제(희소성, 희소 행렬)
    - 단어의 개수는 수만~수십만 인데 비하여(column 수) 각 문장에 나타나는 단어의 수는 적기 때문에 데이터에 0이 많음
    - 희소 행렬은 ML의 성능을 떨어뜨림

## BOW 피처 벡터화
- M개의 텍스트 문서, 총 N개의 단어가 있을 경우 M x N개의 단어 피처로 이뤄진 행렬을 구성
    - column : 단어 feature들
    - row : 각 문서의 단어 빈도/정규화 변환된 횟수
- 카운트 기반의 벡터화
    - 해당 단어가 나타나는 횟수
    - 언어의 특성상 문장에서 자주 사용하는 단어에 높은 값을 부여한다는 단점이 있음
- TF-IDF(Term Frequency Inverse Document Frequency) 벡터화
    - 모든 문서에서 전반적으로 자주 나타나는 단어에 페널티를 주는 방식
    - $\text{TFIDF}_i = \text{TF}_i \times \log{\dfrac{N}{\text{DF}_i}}$
    - $\text{TF}_i$ : 개별 문서에서의 단어 $i$ 빈도
    - $\text{DF}_i$ : 단어 $i$를 가지고 있는 문서 개수
    - $N$ : 전체 문서 개수

## 사이킷런 `CountVectorizer`, `TfidfVectorizer`

- 소문자 일괄 변환, 토큰화, 스톱 워드 필터링 등의 데이터 전처리도 함께 수행
- fit, transform을 통해 객체 반환
- 둘 모두의 파라미터는 동일함

### 파라미터
- max_df : 자주 나타나는 단어를 제외하기 위한 파라미터
    - 정수 값을 주면 그 단어 이하의 단어만 피처로 추출
    - 부동소수 값을 주면 해당 빈도수 이하의 단어만 피처로 추출
- min_df : 너무 적게 나타나는 단어를 제외하기 위한 파라미터 - 가비지성 단어일 확률이 높음
    - 정수 값을 주면 그 단어 이상의 단어만 피처로 추출
    - 부동소수 값을 주면 해당 빈도수 이상의 단어만 피처로 추출
- max_features : 추출하는 피처 개수 제한
    - 가장 높은 빈도를 가지는 단어 순으로 자름
- stop_words : 'english'로 지정하면 영어의 스톱 워드를 추출하지 않음
- n_gram_range : n_gram 범위 설정. 튜플 단위 (범위 최솟값, 범위 최댓값)을 지정
    - (1, 1)로 지정하면 토큰화된 단어를 1개씩 피처로 추출
    - (1, 2)로 지정하면 토큰화된 단어를 1개씩, 순서대로 2개씩 묶어서 피처로 추출
- analyzer : 피처 추출을 수행할 단위. 기본값은 word이며 character의 특정 범위를 피처로 만드는 특정한 경우에 사용
- token_pattern : 토큰화 수행하는 정규 표현식 패턴.
    - 기본값 : '\b\w\w+\b'로 공백 또는 개행 문자 등으로 구분된 단어 분리자(\b) 사이의 두 문자 이상(\w\w+)의 단어를 토큰으로 분리
- tokenizer : 별도의 토큰화 함수를 이용시 적용

### 알고리즘
1. 사전 데이터 가공 : 모든 문자를 소문자로 변경 (lowercase=True)
2. 토큰화 : n_gram_range를 반영하여 각 단어를 토큰화, 단어 기준 (analyzer=True)
3. 텍스트 정규화 : Stop Words 필터링만 수행하며 Stemmer, Lemmatize는 지원되지 않으므로 외부 패키지로 미리 Text Normalization 수행 필요
4. 피처 벡터화

## BOW 벡터화를 위한 희소 행렬
- BOW 형태를 가진 언어 모델의 피처 벡터화의 대부분이 희소 행렬
- 불필요한 0 값이 메모리 공간에 할당되어 메모리 공간이 많이 필요
- 행렬의 크기가 커서 연산 시 시간이 많이 소모됨

## 희소 행렬 - COO 형식
- COO(Coordinate; 좌표) 형식 : 0이 아닌 데이터만 별도의 데이터 배열에 저장하고 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식
- [[3, 0, 1], [0, 2, 0]] → (0, 0), (0, 2), (1, 1) → row : [0, 0, 1], column : [0, 2, 1]

In [1]:
import numpy as np

dense = np.array([[3, 0, 1], [0, 2, 0]])

In [2]:
from scipy import sparse

# 0이 아닌 데이터 추출
data = np.array([3, 1, 2])

# 행, 열 위치를 각각 배열로 생성
row_pos = np.array([0, 0, 1])
col_pos = np.array([0, 2, 1])

# sparse 패키지의 coo_matrix를 이용해 COO 형식으로 희소 행렬 생성
sparse_coo = sparse.coo_matrix((data, (row_pos, col_pos)))

In [4]:
# 원래 형태의 행렬로 전환
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

## 희소 행렬 - CSR 형식
- CSR(Compressed Sparse Row)
- 행 위치 배열 [0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5]를 CSR로 변환하면 [0, 2, 7, 9, 10, 12]로 연속해서 나오는 숫자의 첫번째 위치값만 뽑아낸 뒤 마지막에 데이터의 총 항목 개수를 배열에 추가(이 경우 13)

In [5]:
from scipy import sparse

dense2 = np.array([[0, 0, 1, 0, 0, 5],
                  [1, 4, 0, 3, 2, 5],
                  [0, 6, 0, 3, 0, 0],
                  [2, 0, 0, 0, 0, 0],
                  [0, 0, 0, 7, 0, 8],
                  [1, 0, 0, 0, 0, 0]])

# 0이 아닌 데이터 추출
data2 = np.array([1, 5, 1, 4, 3, 2, 5, 6, 3, 2, 7, 8, 1])

# 행, 열 위치를 각각 배열로 생성
row_pos = np.array([0, 0, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 5])
col_pos = np.array([2, 5, 0, 1, 3, 4, 5, 1, 3, 0, 3, 5, 0])

# COO 형식으로 변환
sparse_coo = sparse.coo_matrix((data2, (row_pos, col_pos)))

# 행 위치 배열의 고유한 값의 시작 위치 인덱스를 배열로 생성
row_pos_ind = np.array([0, 2, 7, 9, 10, 12, 13])

# CRS 형식으로 변환
sparse_csr = sparse.csr_matrix((data2, col_pos, row_pos_ind))

print('COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

COO 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


### 실제로 생성

In [6]:
dense3 = np.array([[0, 0, 1, 0, 0, 5],
                  [1, 4, 0, 3, 2, 5],
                  [0, 6, 0, 3, 0, 0],
                  [2, 0, 0, 0, 0, 0],
                  [0, 0, 0, 7, 0, 8],
                  [1, 0, 0, 0, 0, 0]])

coo = sparse.coo_matrix(dense3)
csr = sparse.csr_matrix(dense3)

print('COO로 변환된 데이터')
print(coo)
print('CSR로 변환된 데이터')
print(csr)

COO로 변환된 데이터
  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1
CSR로 변환된 데이터
  (0, 2)	1
  (0, 5)	5
  (1, 0)	1
  (1, 1)	4
  (1, 3)	3
  (1, 4)	2
  (1, 5)	5
  (2, 1)	6
  (2, 3)	3
  (3, 0)	2
  (4, 3)	7
  (4, 5)	8
  (5, 0)	1
